In [1]:
import socket
import os
from _thread import *
from random import randrange, getrandbits
from math import gcd
import json
import time
import datetime
# python code for fibonacci sequence


In [2]:
def is_prime(n, k=8):
    """ Test if a number is prime
        Args:
            n -- int -- the number to test
            k -- int -- the number of tests to do
        return True if n is prime
    """
    # Test if n is not even.
    # But care, 2 is prime !
    if n == 2 or n == 3:
        return True
    if n <= 1 or n % 2 == 0:
        return False
    # find r and s
    s = 0
    r = n - 1
    while r & 1 == 0:
        s += 1
        r //= 2
    # do k tests
    for _ in range(k):
        a = randrange(2, n - 1)
        x = pow(a, r, n)
        if x != 1 and x != n - 1:
            j = 1
            while j < s and x != n - 1:
                x = pow(x, 2, n)
                if x == 1:
                    return False
                j += 1
            if x != n - 1:
                return False
    return True

def generate_prime_candidate(length):
    """ Generate an odd integer randomly
        Args:
            length -- int -- the length of the number to generate, in bits
        return a integer
    """
    # generate random bits
    p = getrandbits(length)
    # apply a mask to set MSB and LSB to 1
    p |= (1 << length - 1) | 1
    return p
def generate_prime_number(prev, length=8):
    """ Generate a prime
        Args:
            length -- int -- length of the prime to generate, in          bits
        return a prime
    """
    p = 4
    # keep generating while the primality test fail
    while not is_prime(p, 4) or p == prev:
        p = generate_prime_candidate(length)
    return p


In [3]:
def generate_public_key(phi):
    e = 2
    while True:
        if gcd(e, phi) == 1:
            return e
        e += 1

In [4]:
def generate_key():
    p = generate_prime_number(1,8)
    q = generate_prime_number(p,8)
    n = p * q
    phi = (p-1) * (q-1)
    e = generate_public_key(phi)
    d = pow(e, -1, phi)
    return (e, n), (d, n)

In [5]:
def encrypt(pk, plaintext):
    # Unpack the key into it's components
    key, n = pk
    # Convert each letter in the plaintext to numbers based on the character using a^b mod m
    cipher = [pow(ord(char), key, n) for char in plaintext]
    # Return the array of bytes
    return cipher

In [6]:
def decrypt(pk, ciphertext):
    # Unpack the key into its components
    key, n = pk
    # Generate the plaintext based on the ciphertext and key using a^b mod m
    plain = [chr(pow(char, key, n)) for char in ciphertext]
    # Return the array of bytes as a string
    return ''.join(plain)

In [7]:
def generate_id():
    return os.urandom(16).hex()

def get_timestamp():
    return int(time.time())

def get_nonce():
    # generate a random 64-bit number between 2^32 and 2^64-1
    return randrange(2**16)

def get_nonce_acknowledgment(nonce_received):
    # XORing with a constant value twice gives back the original value
    return nonce_received ^ 0xFFFF 

def get_validity_timestamp():
    return get_timestamp() + 5*60*1000

def check_validity_message(valid_till):
    return valid_till > get_timestamp()


In [8]:
def add_client_info(client_name, public_key, client_info_dict, client_id, host_address, port_number):
    client_info_dict[client_name] = {
        "public_key": public_key,
        "client_id": client_id,
        "host_address": host_address,
        "port_number": port_number
    }

def get_client_info(client_name, client_info_dict):
    return client_info_dict[client_name]

In [9]:
def create_message(message_type, message_payload, input_client_name, client_id):
    return {
        'type': message_type,
        'client_name':input_client_name,
        'id': client_id,
        'timestamp': get_timestamp(),
        'valid_till':get_validity_timestamp(),
        'nonce':get_nonce(),
        'message_payload': message_payload
    }

In [10]:
def send_hello_message_to_fellow_server(FellowServerSocket, input_client_name, client_id, sender_public_key, client_server_name):
    message_payload = {
        'client_name':input_client_name,
    }
    fellow_client_hello_message = create_message("fellow_client_hello", message_payload, input_client_name, client_id)
    fellow_client_hello_message_str = json.dumps(fellow_client_hello_message)
    encrypted_message = encrypt(sender_public_key, fellow_client_hello_message_str)
    FellowServerSocket.send(str.encode(json.dumps(encrypted_message)))
    print('\n\n----------- SENDING HELLO MESSAGE FROM FELLOW CLIENT TO FELLOW SERVER -----------')
    now = datetime.datetime.now()
    print('\n[{}] Sent fellow_client_hello message from client {} to fellow server {}, sent message nonce {}'.format(now.strftime("%Y-%m-%d %H:%M:%S:%f"), input_client_name, client_server_name, fellow_client_hello_message["nonce"]))
    return fellow_client_hello_message["nonce"]

In [11]:
def receive_hello_message_from_fellow_client(PKDAServerSocket, FellowClientSocket,input_client_name, client_id, public_key_ring, private_key):
    while True:
        encrypted_fellow_client_message_str = FellowClientSocket.recv(130172).decode('utf-8')
        if not encrypted_fellow_client_message_str:
            break
        else:
            encrypted_fellow_client_message = json.loads(encrypted_fellow_client_message_str)
            decrypted_fellow_client_message = decrypt(private_key, encrypted_fellow_client_message)
            fellow_client_message = json.loads(decrypted_fellow_client_message)
            if not check_validity_message(fellow_client_message["valid_till"]):
                print("Message expired")
                break
            else:
                ## no need to check the nonce here, as the fellow server is receiving a new message from the fellow client
                if(fellow_client_message["type"] == "fellow_client_hello"):
                    client_name = fellow_client_message['message_payload']["client_name"]
                    fellow_client_hello_message_nonce= fellow_client_message["nonce"]
                    print('\n\n----------- FELLOW SERVER RECEIVING HELLO MESSAGE FROM FELLOW CLIENT  -----------')
                    now = datetime.datetime.now()
                    print('\n[{}] Received a request to connect from a fellow client {}, received message nonce {}'.format(now.strftime("%Y-%m-%d %H:%M:%S:%f"), client_name, fellow_client_hello_message_nonce))
                    public_key_request_nonce = send_public_key_request_to_PKDA(PKDAServerSocket, input_client_name, client_id, client_name)
                    receive_public_key_request_from_PKDA(PKDAServerSocket, input_client_name, client_id, public_key_ring, client_name, public_key_request_nonce, private_key, 0)
                    return (fellow_client_hello_message_nonce, client_name)
                else:
                    print("Invalid message type received")
                    break

In [12]:
def receive_hello_message_from_fellow_server(FellowServerSocket,input_client_name, client_id, public_key_ring, private_key, latest_message_nonce):
    while True:
        encrypted_fellow_client_message_str = FellowServerSocket.recv(130172).decode('utf-8')
        if not encrypted_fellow_client_message_str:
            break
        else:
            encrypted_fellow_client_message = json.loads(encrypted_fellow_client_message_str)
            decrypted_fellow_client_message = decrypt(private_key, encrypted_fellow_client_message)
            fellow_client_message = json.loads(decrypted_fellow_client_message)
            if not check_validity_message(fellow_client_message["valid_till"]):
                print("Message expired")
                break
            else:
                acknowledgement_nonce = fellow_client_message['message_payload']['acknowledgement_nonce']
                if latest_message_nonce != get_nonce_acknowledgment(acknowledgement_nonce):
                    print('The server did not acknowledge the message, nonce mismatch')
                else:
                    if(fellow_client_message["type"] == "fellow_client_hello_acknowledgement"):
                        print('\n\n----------- FELLOW CLIENT RECEIVING HELLO ACKNOWLEDGEMENT MESSAGE FROM FELLOW SERVER  -----------')
                        now = datetime.datetime.now()
                        print('\n[{}] Received an acknowledgment from fellow server {}, for message nonce {}. Received message nonce {}'.format(now.strftime("%Y-%m-%d %H:%M:%S:%f"), fellow_client_message['client_name'], get_nonce_acknowledgment(acknowledgement_nonce), fellow_client_message['nonce']))
                        return fellow_client_message['nonce']
                    else:
                        print("Invalid message type received")
                        break

In [13]:
def send_final_acknowledgement_to_fellow_server(FellowServerSocket, input_client_name, client_id, acknowledgement_nonce, client_public_key, fellow_server_name):
    message_payload = {
        'acknowledgement_nonce':get_nonce_acknowledgment(acknowledgement_nonce)
    }
    fellow_client_hello_acknowledgment_message = create_message("fellow_client_final_acknowledgment", message_payload, input_client_name, client_id)
    fellow_client_hello_acknowledgment_message_str = json.dumps(fellow_client_hello_acknowledgment_message)
    encrypted_message = encrypt(client_public_key, fellow_client_hello_acknowledgment_message_str)
    FellowServerSocket.send(str.encode(json.dumps(encrypted_message)))
    print('\n\n----------- SENDING FINAL ACKNOWLEDGEMENT MESSAGE FROM FELLOW CLIENT TO FELLOW SERVER -----------')
    now = datetime.datetime.now()
    print('\n[{}] Sending final acknowledgment from fellow client {} to fellow server {}, sent message nonce: {}'.format(now.strftime("%Y-%m-%d %H:%M:%S:%f"), input_client_name, fellow_server_name ,fellow_client_hello_acknowledgment_message["nonce"]))
    return fellow_client_hello_acknowledgment_message["nonce"]

In [14]:
def fellow_client_talk_thread(input_client_name, requested_client_info, client_id, private_key, public_key_ring, requested_client_name):
    FellowServerSocket = socket.socket()
    try:
        FellowServerSocket.connect((requested_client_info['host_address'], requested_client_info['port_number']))
    except socket.error as e:
        print(str(e))
    
    hello_message_nonce = send_hello_message_to_fellow_server(FellowServerSocket, input_client_name, client_id, requested_client_info['public_key'], requested_client_name)
    server_hello_message_nonce = receive_hello_message_from_fellow_server(FellowServerSocket, input_client_name, client_id, public_key_ring, private_key,hello_message_nonce)
    send_final_acknowledgement_to_fellow_server(FellowServerSocket, input_client_name, client_id, server_hello_message_nonce, requested_client_info['public_key'], requested_client_name)
    return 

In [15]:
def send_public_key_request_to_PKDA(PKDAServerSocket, input_client_name, client_id, inputFromUser):
    message_payload = {
        'requested_client': inputFromUser,
    }
    client_request_message = create_message('public_key_request', message_payload ,input_client_name, client_id)
    client_request_message_str = json.dumps(client_request_message)
    PKDAServerSocket.send(str.encode(client_request_message_str))
    print('\n\n----------- SENDING PUBLIC KEY REQUEST MESSAGE FROM CLIENT TO PKDA SERVER -----------')
    now = datetime.datetime.now()
    print('\n[{}] Sending a public key request from client {} to PKDA server requesting public key for client {}, sent message nonce: {}'.format(now.strftime("%Y-%m-%d %H:%M:%S:%f"), input_client_name, inputFromUser, client_request_message['nonce']))
    return client_request_message['nonce']

In [16]:
def receive_public_key_request_from_PKDA(PKDAServerSocket, input_client_name, client_id, public_key_ring, inputFromUser, latest_message_nonce, private_key, connect):
    while True:
        server_response_message_str = PKDAServerSocket.recv(131072).decode('utf-8')
        if not server_response_message_str:
            break
        else:
            encrypted_server_response = json.loads(server_response_message_str)
            pkda_info = get_client_info('pkda', public_key_ring)
            decrypted_message = decrypt(pkda_info['public_key'], encrypted_server_response)
            server_response_message = json.loads(decrypted_message)
            if not check_validity_message(server_response_message['valid_till']):
                print('Invalid message, message expired')
            else:
                acknowledgement_nonce = server_response_message['message_payload']['acknowledgement_nonce']
                if latest_message_nonce != get_nonce_acknowledgment(acknowledgement_nonce):
                    print('The server did not acknowledge the public key request message, nonce mismatch')
                else:
                    print('----------- PKDA SERVER RECEIVING PUBLIC KEY REQUEST MESSAGE-----------')
                    now = datetime.datetime.now()
                    print('\n[{}] PKDA Server responded to the public key request with message nonce {}'.format(now.strftime("%Y-%m-%d %H:%M:%S:%f"), get_nonce_acknowledgment(acknowledgement_nonce)))
                    response_payload = server_response_message['message_payload']
                    requested_client_info = response_payload['requested_client_info']
                    add_client_info(inputFromUser, requested_client_info['public_key'], public_key_ring, requested_client_info['client_id'], requested_client_info['host_address'], requested_client_info['port_number'])
                    print('Client info added to {}\'s public key ring client_name: {}, client_id:{}, client_host_address:{}, client_port_number:{}'.format(input_client_name,inputFromUser,requested_client_info['client_id'], requested_client_info['host_address'], requested_client_info['port_number']))
                    if connect:
                        start_new_thread(fellow_client_talk_thread, (input_client_name, requested_client_info, client_id, private_key, public_key_ring, inputFromUser ))
                    break

In [17]:
def send_hello_acknowledgment_to_fellow_client(FellowClientSocket, input_client_name, client_id, public_key_ring, acknowledgement_nonce, client_name):
    message_payload = {
        'acknowledgement_nonce': get_nonce_acknowledgment(acknowledgement_nonce) #this nonce is a well-known function of nonce (f(N1)) of the hello message
    }
    fellow_client_hello_acknowledgment_message = create_message("fellow_client_hello_acknowledgement", message_payload, input_client_name, client_id)
    fellow_client_hello_acknowledgment_message_str = json.dumps(fellow_client_hello_acknowledgment_message)
    encrypted_message = encrypt(public_key_ring[client_name]['public_key'], fellow_client_hello_acknowledgment_message_str)
    FellowClientSocket.send(str.encode(json.dumps(encrypted_message)))
    print('----------- SENDING HELLO ACKNOWLEDGMENT MESSAGE FROM FELLOW SERVER TO FELLOW CLIENT -----------')
    now = datetime.datetime.now()
    print('\n[{}] Sending hello acknowledgment to fellow client {} from fellow server {}, sent message nonce: {}'.format(now.strftime("%Y-%m-%d %H:%M:%S:%f"), client_name, input_client_name ,fellow_client_hello_acknowledgment_message["nonce"]))
    return fellow_client_hello_acknowledgment_message["nonce"]

In [18]:
def receive_final_acknowledgement_from_fellow_client(FellowClientSocket, private_key, latest_message_nonce):
    while True:
        encrypted_fellow_client_message_str = FellowClientSocket.recv(130172).decode('utf-8')
        if not encrypted_fellow_client_message_str:
            break
        else:
            encrypted_fellow_client_message = json.loads(encrypted_fellow_client_message_str)
            decrypted_fellow_client_message = decrypt(private_key, encrypted_fellow_client_message)
            fellow_client_message = json.loads(decrypted_fellow_client_message)
            if not check_validity_message(fellow_client_message["valid_till"]):
                print("Message expired")
                break
            else:
                acknowledgement_nonce = fellow_client_message['message_payload']['acknowledgement_nonce']
                if latest_message_nonce != get_nonce_acknowledgment(acknowledgement_nonce):
                    print('The fellow client did not acknowledge the message, nonce mismatch')
                else:
                    if(fellow_client_message["type"] == "fellow_client_final_acknowledgment"):
                        print('-----------FELLOW SERVER RECEIVING FINAL ACKNOWLEDGMENT MESSAGE FROM FELLOW CLIENT -----------')
                        now = datetime.datetime.now()
                        print('[{}] Received final acknowledgment from fellow client {} for message nonce {}'.format(now.strftime("%Y-%m-%d %H:%M:%S:%f"), fellow_client_message['client_name'],get_nonce_acknowledgment(acknowledgement_nonce)))
                    else:
                        print("Invalid message type received")
                    break

In [19]:
def fellow_server_talk_thread(PKDAServerSocket, FellowClientSocket, input_client_name, client_id, public_key_ring , private_key):
    (fellow_client_hello_message_nonce, client_name) = receive_hello_message_from_fellow_client(PKDAServerSocket, FellowClientSocket, input_client_name, client_id, public_key_ring, private_key)
    fellow_client_acknowledgement_nonce =  send_hello_acknowledgment_to_fellow_client(FellowClientSocket, input_client_name, client_id, public_key_ring, fellow_client_hello_message_nonce, client_name)
    receive_final_acknowledgement_from_fellow_client(FellowClientSocket, private_key, fellow_client_acknowledgement_nonce)
    return 
                
                    

In [20]:
def send_handshake_to_PKDA(PKDAServerSocket, input_client_name, id, public_key, host, self_port):
    message_payload = {
        "public_key": public_key,
        "client_host_address": host,
        "client_port_number": self_port,
    }
    client_handshake_message = create_message('handshake', message_payload, input_client_name, id)

    client_handshake_message_str = json.dumps(client_handshake_message)
    PKDAServerSocket.send(str.encode(client_handshake_message_str))
    return client_handshake_message['nonce']

In [21]:
def receive_handshake_from_PKDA(PKDAServerSocket, public_key_ring, latest_message_nonce, host, port):
    server_handshake_message_str = PKDAServerSocket.recv(2048).decode('utf-8')
    server_handshake_message = json.loads(server_handshake_message_str)

    if not check_validity_message(server_handshake_message['valid_till']):
        print('Invalid message, message expired')
    else:
        acknowledgement_nonce = server_handshake_message['message_payload']['acknowledgement_nonce']
        if latest_message_nonce != get_nonce_acknowledgment(acknowledgement_nonce):
            print('The server did not acknowledge the handshake message, nonce mismatch')
        else:
            print('Handshake successful')
            add_client_info('pkda', server_handshake_message['message_payload']["public_key"], public_key_ring, server_handshake_message["id"], host, port)

In [22]:
def PKDA_server_talk_thread(PKDAServerSocket, client_id, input_client_name, public_key_ring, private_key):
    while True:
        inputFromUser = input('Input client name for connecting : ')
        if inputFromUser == 'exit' or inputFromUser == '':
            break
        else:
            public_key_request_nonce = send_public_key_request_to_PKDA(PKDAServerSocket, input_client_name, client_id, inputFromUser)
            receive_public_key_request_from_PKDA(PKDAServerSocket, input_client_name, client_id, public_key_ring, inputFromUser, public_key_request_nonce, private_key, 1)

In [23]:
PKDAServerSocket = socket.socket()
host = '127.0.0.1'
port = 3000
# generate random number greater than 1024 for the port number
self_port = 3000
while self_port <= 1024 or self_port == 3000: 
    self_port = randrange(2**16)
print("Self port number: ", self_port)
input_client_name = input('Input your client name for identification: ')
public_key, private_key = generate_key()
id = generate_id()
public_key_ring = {}
## public_key_ring structure
# {
#     'client_name': { # unique string name of the client
#         'host_address': host_address, # host address of the client
#         'port_number': port, # port of the client to connect to
#         'public_key': public_key, # public key of the client
#         'id': client_id # unique 16 byte id of the client
#     }
# }
try:
    PKDAServerSocket.connect((host, port))
except socket.error as e:
    print(str(e))

## message structure
# {
#     "type":'handshake', # type of message
#     'id':'', # unique id of the sender
#     'client_name': '', # name of the sender
#     'public_key': '', # public key of the sender
#     'timestamp': '', # timestamp of the message
#     'nonce': '', # nonce of the message
#     'valid_till': '', # validity timestamp of the message
#     'acknowledgement_nonce': '' # f(previous_message_nonce) XORed with a constant value
# }

## first message in the channel 
handshake_nonce = send_handshake_to_PKDA(PKDAServerSocket, input_client_name, id, public_key, host, self_port)
receive_handshake_from_PKDA(PKDAServerSocket, public_key_ring, handshake_nonce, host, port)

FellowClientSocket = socket.socket()
try:
    FellowClientSocket.bind((host, self_port))
except socket.error as e:
    print(str(e))

print('Client is accepting connections now....')
FellowClientSocket.listen(5)
start_new_thread(PKDA_server_talk_thread, (PKDAServerSocket, id, input_client_name, public_key_ring, private_key))
while True:
    Client, address = FellowClientSocket.accept()
    print('Connected to fellow client: ' + address[0] + ':' + str(address[1]))
    start_new_thread(fellow_server_talk_thread, (PKDAServerSocket, Client, input_client_name, id ,public_key_ring, private_key))


Self port number:  58745
167 163
Handshake successful
Client is accepting connections now....
Server responded to the public key request 
Client info added to the public key ring d83181eec9ac0f0cce1d42be0ee9daf3 127.0.0.1 14157


Exception ignored in thread started by: <function fellow_client_talk_thread at 0x0000029BBCF37740>
Traceback (most recent call last):
  File "C:\Users\SAATVIK\AppData\Local\Temp\ipykernel_29960\3852966038.py", line 9, in fellow_client_talk_thread
TypeError: receive_hello_message_from_fellow_server() missing 2 required positional arguments: 'private_key' and 'latest_message_nonce'


Sent fellow_client_hello message to fellow server
